In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

In [3]:
spark = (
    SparkSession
    .builder
    .appName('Yello Taxi')
    .master('local[*]')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/05 09:53:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.stop()

In [4]:
spark

## Q1. Read the data for January. How many columns are there?

In [5]:
df = spark.read.parquet('data/yellow_tripdata_2023-01.parquet')

In [6]:
len(df.columns)

19

## Answer: 19

## Q2. What's the standard deviation of the trips duration in January?

In [7]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee']

In [8]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [9]:
df.select('tpep_pickup_datetime', 'tpep_dropoff_datetime').show()

+--------------------+---------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|
+--------------------+---------------------+
| 2023-01-01 00:32:10|  2023-01-01 00:40:36|
| 2023-01-01 00:55:08|  2023-01-01 01:01:27|
| 2023-01-01 00:25:04|  2023-01-01 00:37:49|
| 2023-01-01 00:03:48|  2023-01-01 00:13:25|
| 2023-01-01 00:10:29|  2023-01-01 00:21:19|
| 2023-01-01 00:50:34|  2023-01-01 01:02:52|
| 2023-01-01 00:09:22|  2023-01-01 00:19:49|
| 2023-01-01 00:27:12|  2023-01-01 00:49:56|
| 2023-01-01 00:21:44|  2023-01-01 00:36:40|
| 2023-01-01 00:39:42|  2023-01-01 00:50:36|
| 2023-01-01 00:53:01|  2023-01-01 01:01:45|
| 2023-01-01 00:43:37|  2023-01-01 01:17:18|
| 2023-01-01 00:34:44|  2023-01-01 01:04:25|
| 2023-01-01 00:09:29|  2023-01-01 00:29:23|
| 2023-01-01 00:33:53|  2023-01-01 00:49:15|
| 2023-01-01 00:13:04|  2023-01-01 00:22:10|
| 2023-01-01 00:45:11|  2023-01-01 01:07:39|
| 2023-01-01 00:04:33|  2023-01-01 00:19:22|
| 2023-01-01 00:03:36|  2023-01-01 00:09:36|
| 2023-01-

In [10]:
df_with_duration = df.select(
    *df.columns,
    ((
        F.col('tpep_dropoff_datetime') - F.col('tpep_pickup_datetime')
    ).cast('long') / 60).alias('duration')
)

In [11]:
df_with_duration.select(F.stddev('duration')).show()

+------------------+
|  stddev(duration)|
+------------------+
|42.594351241955756|
+------------------+



## Answer: 42.59

## Q3. Dropping outliers (keep only the records where the duration was between 1 and 60 minutes (inclusive))

In [12]:
current_rown = df_with_duration.count()
current_rown

3066766

In [13]:
df_no_outliers = df_with_duration.filter((F.col('duration') > 1) & (F.col('duration') <= 60))

In [14]:
df_no_outliers.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|          duration|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       2| 2023-01-01 00:32:10|  2023-01-01 00:40:36|            1.0|         0.97|       1.0|                 N|         161|    

## What fraction of the records left after you dropped the outliers?

In [15]:
df_no_outliers.count() / current_rown * 100

98.11286547457485

## ANSWER: 98%

## Q4. One-hot encoding

In [16]:
ohe = OneHotEncoder(inputCols=["PULocationID", "DOLocationID"], outputCols=["PU_OHE", "DO_OHE"])
assembler = VectorAssembler(inputCols=["PU_OHE", "DO_OHE"], outputCol="features")

In [17]:
ohe_model = ohe.fit(df_no_outliers)

In [18]:
train_df = assembler.transform(ohe_model.transform(df_no_outliers))

In [19]:
train_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-----------------+-----------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|          duration|           PU_OHE|           DO_OHE|            features|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+-----------------

## ANSWER: 530 (close to 515)

## Q5. Training a model

In [20]:
lr = LinearRegression(featuresCol="features", labelCol="duration")

In [21]:
lr_model = lr.fit(train_df)

24/06/04 20:09:22 WARN Instrumentation: [3b1b2cbc] regParam is zero, which might cause numerical instability and overfitting.
24/06/04 20:09:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/04 20:09:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
24/06/04 20:09:30 WARN Instrumentation: [3b1b2cbc] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [22]:
train_predictions = lr_model.transform(train_df)

In [23]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="duration", metricName="rmse")

In [24]:
evaluator.evaluate(train_predictions)

7.64823034523142

## ANSWER: 7.64

## Q6. Evaluating the model

In [25]:
test_df = spark.read.parquet('data/yellow_tripdata_2023-02.parquet')

In [26]:
test_df = test_df.select(
    *test_df.columns,
    ((
        F.col('tpep_dropoff_datetime') - F.col('tpep_pickup_datetime')
    ).cast('long') / 60).alias('duration')
).filter((F.col('duration') > 1) & (F.col('duration') <= 60))

In [27]:
test_df = assembler.transform(ohe_model.transform(test_df))

In [28]:
test_predictions = lr_model.transform(test_df)

In [30]:
evaluator.evaluate(test_predictions)

7.810105882139033

## ANSWER: 7.81

In [31]:
spark.stop()